## Preparación de datos para sistema recomendador proxy

En este *notebook* prepararemos los datos necesarios para evaluar un sistema recomendador externo con la librería `Elliot`. Se usarán los datasets de libros, usuarios y ratings.

### Creación de datasets de entrenamiento, validación y test

In [1]:
import os
import pandas as pd

# Ruta de datasets procesados
ready_path = os.path.join(os.getcwd(), "../..", "datasets", "ready")

# Lectura del dataset de ratings procesado
col_types = {
    'user_id': 'int32',
    'book_id': 'int32',
    'rating': 'float32',
}
ratings_df = pd.read_csv(ready_path + "/ratings.csv", dtype=col_types)
print("# de ratings:", ratings_df.shape[0])

# de ratings: 5515602


In [2]:
# Ruta de datasets en bruto
raw_path = os.path.join(os.getcwd(), "../..", "datasets", "raw")

# Lectura del dataset de libros en bruto (coincidirá con el procesado después de limpiarlo)
books_df = pd.DataFrame(pd.read_pickle(raw_path + "/books_raw.pkl"))
print("# de libros:", books_df.shape[0])

# de libros: 9467


Separamos el dataset de ratings en parte de entrenamiento y parte de test, en una proporción del 80% y 20%, respectivamente.

In [3]:
# Creamos los datasets de train (80%) y test (20%)
import sklearn.model_selection as model_selection

train, test = model_selection.train_test_split(ratings_df, test_size=0.2, random_state=42)
print("# de ratings en train (train + validation):", train.shape[0])
print("# de ratings en test:", test.shape[0])

# de ratings en train (train + validation): 4412481
# de ratings en test: 1103121


Además, en vista de modelos futuros, el dataset de entrenamiento será parte entrenamiento (70% del global) y parte validación (10% del global). Es fácil comprobar que se debe hacer un split local del dataset de entrenamiento en un 87.5% para entrenamiento *per se* y un 12.5% para validación, consiguiendo los porcentajes globales deseados.

In [4]:
# Creamos los datasets de train (87.5% -> 70% del global) y validación (12.5% -> 10% del global)
train_train, train_valid = model_selection.train_test_split(train, test_size=0.125, random_state=42)
print("# de ratings en train:", train_train.shape[0])
print("# de ratings en validation:", train_valid.shape[0])

# de ratings en train: 3860920
# de ratings en validation: 551561


### Entrenamiento del modelo (perfiles de usuario)

Para el modelo, crearemos un dataset de perfiles de usuario que codifique sus preferencias a partir del conjunto de datos de entrenamiento. Los perfiles de usuario estarán basados en los libros que haya puntuado. En concreto, se tendrán las mismas columnas que con los libros: `semantic_sbert`, `semantic_use` y `sentiment`. Pero, en este caso, cada una de las columnas corresponderá a una ponderación calculada de esta manera:

Tomaremos aquellos libros que haya valorado positivamente, es decir, con una puntuación mayor o igual que 0.75. Sea entonces $i_k$ el identificador del k-ésimo libro puntuado por el usuario de ese subconjunto de libros. El vector resultante es el normalizado de $$\frac{\sum_{k}^{}rating(i_k) * \vec{v}_{libro}(i_k)}{\sum_{k}^{}rating(i_k)}$$

In [5]:
# Creamos el dataframe de entrenamiento para el modelo a partir del dataset de train
train_df = pd.DataFrame(train, columns=['user_id', 'book_id', 'rating'])
train_df.sort_values(by=['user_id', 'book_id'], inplace=True)
train_df.reset_index(drop=True, inplace=True)
train_df.head()

,user_id,book_id,rating
0,1,11,1.00
1,1,13,0.75
2,1,31,0.75
3,1,32,0.75
4,1,33,0.75


In [6]:
# Creamos un dataframe con los usuarios que aparecen en el dataset de train
users_df = pd.DataFrame(columns=['user_id'])
users_df['user_id'] = train_df['user_id'].unique()
users_df.sort_values(by=['user_id'], inplace=True)
users_df.reset_index(drop=True, inplace=True)
print("# de usuarios:", users_df.shape[0])

# de usuarios: 52371


In [7]:
# Obtenemos la información de los libros que han sido valorados por los usuarios del dataset de train
books_df = books_df[books_df['book_id'].isin(train_df['book_id'].unique())]
books_df.sort_values(by=['book_id'], inplace=True)
books_df.reset_index(drop=True, inplace=True)
print("# de libros:", books_df.shape[0])

# de libros: 9467


In [8]:
import numpy as np

def create_user_feature(user_id, feature, train_df: pd.DataFrame, books_df: pd.DataFrame):
    """
    Crea un vector normalizado de la característica dada a partir
    de los libros que al usuario le gustaron (rating >= 0.75). El vector
    es la suma ponderada de los vectores de la característica dada de
    dichos libros, donde los pesos son los ratings de los libros.

    ## Parámetros
    - user_id: ID del usuario.
    - param feature: Nombre de la característica (columna) a calcular.
    - train_df: `DataFrame` del dataset de entrenamiento.
    - books_df: `DataFrame` de libros en el dataset de entrenamiento.

    ## Retorna
    Vector normalizado de la característica dada.
    """
    # Ratings positivas del usuario
    user_ratings = train_df[train_df['user_id'] == user_id]
    user_likes = user_ratings[user_ratings['rating'] >= 0.75]
    # Libros que le gustaron al usuario
    user_books = books_df[books_df['book_id'].isin(user_likes['book_id'])]
    feature_array = np.array(user_books[feature])
    # Media ponderada de los vectores de la característica dada de los libros
    user_feature = np.sum([feature_array[i]*np.array(user_likes['rating'])[i] for i in range(len(feature_array))], axis=0)
    total_ratings_sum = np.sum(np.array(user_likes['rating']))
    if total_ratings_sum != 0:
        user_feature = user_feature/total_ratings_sum
    # Se retorna el vector normalizado
    norm = np.linalg.norm(user_feature)
    if norm == 0:
        return user_feature
    return user_feature/norm

In [9]:
# Ejemplo de creación de perfil de usuario (ID 1, característica 'sentiment')
create_user_feature(1, 'sentiment', train_df, books_df)

array([0.48328285, 0.02516897, 0.00446195, 0.01960389, 0.26584741,
       0.01181137, 0.0063063 , 0.01262651, 0.01212371, 0.06401394,
       0.01505276, 0.00546003, 0.00681106, 0.01059581, 0.00963669,
       0.00372715, 0.00863336, 0.01646627, 0.01775028, 0.00464807,
       0.80488975, 0.03050896, 0.00974535, 0.11301331, 0.00746177,
       0.01864082, 0.15745456, 0.04579422])

In [10]:
# Característica semantic_sbert de los usuarios
users_df['semantic_sbert'] = users_df['user_id'].apply(lambda x: create_user_feature(x, 'semantic_sbert', train_df, books_df))

In [11]:
# Característica semantic_use de los usuarios
users_df['semantic_use'] = users_df['user_id'].apply(lambda x: create_user_feature(x, 'semantic_use', train_df, books_df))

In [12]:
# Característica sentiment de los usuarios
users_df['sentiment'] = users_df['user_id'].apply(lambda x: create_user_feature(x, 'sentiment', train_df, books_df))

Para posibles usos recurrentes, guardamos nuestro modelo KNN, ya entrenado, en almacenamiento local. Realmente se estará guardando un dataset de perfiles de usuario obtenido a partir del 80% del dataset de ratings correspondiente al entrenamiento.

In [14]:
# Ruta de almacenamiento de modelos
models_path = os.path.join(os.getcwd(), "../..", "models")

# Guardamos el modelo KNN entrenado (dataset de usuarios)
users_df.to_pickle(models_path + "/knn_users.pkl")

Podemos utilizar esta celda para volver a cargar el modelo entrenado, sin necesidad de pasar por el entrenamiento de nuevo.

In [10]:
models_path = os.path.join(os.getcwd(), "../..", "models")
users_df = pd.DataFrame(pd.read_pickle(models_path + "/knn_users.pkl"))

### Predicciones a partir del modelo

Para hacer las predicciones, necesitamos crear una función de similitud que el modelo utilizará para encontrar los k libros más similares en función del perfil de usuario. Como tanto los libros como los perfiles de usuario cuentan con los mismos atributos (`semantic_sbert`, `semantic_use` y `sentiment`) la función de similitud podrá utilizarse para comparar libros con libros, usuarios con usuarios o libros con usuarios (y viceversa).

La fórmula propuesta es la siguiente: $$S(x, y) = w * S_C(x_{sem}, y_{sem}) + (1 - w) * S_C(x_{sent}, y_{sent}),$$ donde $w \in [0, 1]$ es un peso para la ponderación entre la parte semántica y la parte de sentiment analysis y $S_C$ es la similitud coseno entre los vectores.

In [11]:
# Definimos la función de similitud entre dos objetos
def sem_sent_sim(item1, item2, sem_option='semantic_sbert', sem_w=0.75) -> float:
    """
    Calcula la similitud entre un objeto y otro.
    Los objetos deben tener un vector `'semantic'` y otro `'sentiment'`, 
    previamente normalizados.

    La similitud sigue la siguiente fórmula:

    sim(item1, item2) = sem_w * cos_sim(item1[sem_option], item2[sem_option]) + 
    (1 - sem_w) * cos_sim(item1['sentiment'], item2['sentiment'])

    ## Parámetros:
    - item1: Primer objeto.
    - item2: Segundo objeto.
    - sem_option: Opción de contenido semántico. Por defecto su valor
    es `'semantic_sbert'` (modelo SBERT), pero también puede valer `'semantic_use'`
    (modelo USE).
    - sem_w: Peso del contenido semántico. Por defecto su valor es 0.75.

    ## Retorna:
    La similitud entre ambos objetos.
    """
    # Obtenemos el campo semántico de cada objeto
    sem1 = item1[sem_option]
    sem2 = item2[sem_option]
    # Obtenemos el campo de sentimiento de cada objeto
    sent1 = item1['sentiment']
    sent2 = item2['sentiment']
    # Calculamos la similitud
    return sem_w * np.dot(sem1, sem2) + (1 - sem_w) * np.dot(sent1, sent2)

def sem_sent_dist(item1, item2, sem_option='semantic_sbert', sem_w=0.75) -> float:
    """
    Calcula la distancia entre un objeto y otro.

    Los objetos deben tener un vector 'semantic' y otro 'sentiment', 
    previamente normalizados.

    La similitud sigue la siguiente fórmula:

    dist(item1, item2) = 1 - sim(item1, item2)

    ## Parámetros:
    - item1: Primer objeto.
    - item2: Segundo objeto.
    - sem_option: Opción de contenido semántico. Por defecto su valor
    es `'semantic_sbert'` (modelo SBERT), pero también puede valer `'semantic_use'`
    (modelo USE).
    - sem_w: Peso del contenido semántico. Por defecto su valor es 0.75.

    ## Retorna:
    La distancia entre ambos objetos.
    """
    return 1 - sem_sent_sim(item1, item2, sem_option, sem_w)

In [12]:
def k_nearest(item, train_df: pd.DataFrame, books_df: pd.DataFrame, k=10, sem_option='semantic_sbert', sem_w=0.75):
    """
    Calcula los k vecinos más cercanos a un objeto.

    ## Parámetros:
    - item: Objeto del que queremos obtener los k vecinos más cercanos.
    - train_df: `DataFrame` del dataset de entrenamiento.
    - books_df: `DataFrame` de libros en el dataset de entrenamiento.
    - k: Número de vecinos más cercanos que queremos obtener.
    - sem_option: Opción de contenido semántico. Por defecto su valor
    es `'semantic_sbert'` (modelo SBERT), pero también puede valer `'semantic_use'`
    (modelo USE).
    - sem_w: Peso del contenido semántico. Por defecto su valor es 0.75.

    ## Retorna:
    k tuplas (book_id, similitud) con los k vecinos más cercanos al objeto.
    """
    if 'user_id' in item:
        # Ratings positivas del usuario
        user_ratings = train_df[train_df['user_id'] == item['user_id']]
        user_likes = user_ratings[user_ratings['rating'] >= 0.75]
        # Libros que le gustaron al usuario
        user_books = books_df[books_df['book_id'].isin(user_likes['book_id'])]
        # Eliminamos los libros que ya ha valorado el usuario positivamente
        books_df = books_df[~books_df['book_id'].isin(user_books['book_id'])]
    # Calculamos las distancias entre el objeto y todos los demás
    sim_items = books_df.apply(lambda x: sem_sent_sim(item, x, sem_option, sem_w), axis=1)
    # Obtenemos los k vecinos más cercanos
    nearest = sim_items.sort_values(ascending=False).head(k)
    return list(nearest.items())

In [13]:
users_df.head()

,user_id,semantic_sbert,semantic_use,sentiment
0,1,"[0.014834422, -0.05901133, 0.009754823, 0.0025...","[-0.010530782, -0.02715664, 0.026076965, 0.014...","[0.4832828518025987, 0.02516897416610438, 0.00..."
1,2,"[-0.010421482, -0.08042563, 0.028446263, 0.006...","[-0.0072992807, -0.052955978, 0.022262562, 0.0...","[0.32581426562332383, 0.017087785672607495, 0...."
2,4,"[0.007222683, -0.08084607, 0.009723243, 0.0145...","[-0.015179278, -0.020291489, 0.019601513, -0.0...","[0.353391686031033, 0.020578294242811265, 0.00..."
3,5,"[0.0077247894, -0.06979298, 0.017557345, 0.003...","[-0.013398148, 0.00071242295, -0.0065015736, -...","[0.1504242339339517, 0.03044100233742304, 0.00..."
4,6,"[-0.00026792276, -0.05772841, 0.032648098, -0....","[-0.011756575, -0.012661858, 0.03165779, -0.00...","[0.3616514716979118, 0.07584046852229086, 0.00..."


In [14]:
# Ejemplo de obtención de los 10 libros más cercanos a un usuario (ID 1)
nearest = k_nearest(users_df[users_df['user_id'] == 1].iloc[0], train_df, books_df, k=20, sem_w = 1.0)
nearest.sort(key=lambda x: x[0])
nearest

[(235, 0.7780513167381287),
 (639, 0.7824853658676147),
 (1010, 0.7869187593460083),
 (1126, 0.8295992612838745),
 (1260, 0.7990720272064209),
 (1973, 0.7757652997970581),
 (2196, 0.7754789590835571),
 (2547, 0.7897970676422119),
 (2813, 0.7749426960945129),
 (2983, 0.8316555619239807),
 (3093, 0.7934227585792542),
 (3764, 0.7749254107475281),
 (4535, 0.7751350402832031),
 (5460, 0.7788273096084595),
 (5463, 0.787928581237793),
 (7075, 0.7764295339584351),
 (7139, 0.7835315465927124),
 (7805, 0.7797619104385376),
 (7927, 0.7808305621147156),
 (8271, 0.7905234098434448)]

In [30]:
# Calculamos los k libros más similares (de todo el dataset de libros) a cada perfil de usuario (sin contar los ya positivamente valorados)
k = 50
predictions_df = users_df.copy()
predictions_df = predictions_df.drop(columns=['semantic_sbert', 'semantic_use', 'sentiment'])
predictions_df['nearest'] = predictions_df['user_id'].apply(lambda x: k_nearest(users_df[users_df['user_id'] == x].iloc[0], train_df, books_df, k=k, sem_w=0.0))

In [31]:
predictions_df.head()

,user_id,nearest
0,1,"[(9367, 0.995852848061811), (1888, 0.995787125..."
1,2,"[(3513, 0.9885944450523814), (4173, 0.98775853..."
2,4,"[(6290, 0.9947663144349225), (7764, 0.99411586..."
3,5,"[(603, 0.9885972751223394), (7618, 0.987279488..."
4,6,"[(1888, 0.9842987889038904), (6290, 0.98335498..."


In [32]:
# Transformamos la columna 'nearest' en dos columnas: 'book_id' y 'prediction'
predictions_df = predictions_df.explode('nearest')
predictions_df[['book_id', 'prediction']] = pd.DataFrame(predictions_df['nearest'].tolist(), index=predictions_df.index)
predictions_df = predictions_df.drop('nearest', axis=1)

In [33]:
predictions_df.head()

,user_id,book_id,prediction
0,1,9367,0.995853
0,1,1888,0.995787
0,1,362,0.993473
0,1,5545,0.991837
0,1,8623,0.991676


### Creación de los ficheros para Elliot

Una vez calculadas todas las predicciones de los libros por usuario a partir del modelo, guardaremos todos los datasets (entrenamiento, validación y test) así como las propias predicciones en formato .tsv, de tal forma que puedan ser utilizados en `Elliot`.

In [34]:
# Ruta de datasets de entrenamiento
training_path = os.path.join(os.getcwd(), "../..", "datasets", "training")
# Guardamos el dataset de predicciones así como los de entrenamiento, test y validación
predictions_df.to_csv(training_path + "/predictions/predictions_cb_0_100.tsv", sep='\t', header=None, index=False)

In [23]:
# Creamos el dataframe del dataset de test (para darle un formato ordenado)
test_df = pd.DataFrame(test, columns=['user_id', 'book_id', 'rating'])
test_df.sort_values(by=['user_id', 'book_id'], inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [24]:
# Creamos el dataframe de entrenamiento 70% (para darle un formato ordenado)
train_train_df = pd.DataFrame(train_train, columns=['user_id', 'book_id', 'rating'])
train_train_df.sort_values(by=['user_id', 'book_id'], inplace=True)
train_train_df.reset_index(drop=True, inplace=True)

In [25]:
# Creamos el dataframe de validación 10% (para darle un formato ordenado)
train_valid_df = pd.DataFrame(train_valid, columns=['user_id', 'book_id', 'rating'])
train_valid_df.sort_values(by=['user_id', 'book_id'], inplace=True)
train_valid_df.reset_index(drop=True, inplace=True)

In [26]:
train_df.to_csv(training_path + "/train.tsv", sep='\t', header=None, index=False)
test_df.to_csv(training_path + "/test.tsv", sep='\t', header=None, index=False)
train_train_df.to_csv(training_path + "/train_train.tsv", sep='\t', header=None, index=False)
train_valid_df.to_csv(training_path + "/train_valid.tsv", sep='\t', header=None, index=False)

También podemos intentar predecir la nota que daría un usuario a los libros que ya ha valorado a partir del modelo, mediante la métrica de similitud. Esto sería enfocar el modelo hacia una versión *collaborative-filtering*. Adjuntamos una columna `prediction` al dataset total de ratings.

In [35]:
ratings_df['prediction'] = ratings_df.apply(lambda x: sem_sent_sim(users_df[users_df['user_id'] == x['user_id']].iloc[0], books_df[books_df['book_id'] == x['book_id']].iloc[0], sem_w=0.0), axis=1)

In [36]:
ratings_df.head()

,user_id,book_id,rating,prediction
0,1,4,1.00,0.750543
1,1,11,1.00,0.793749
2,1,13,0.75,0.913112
3,1,22,0.50,0.769342
4,1,31,0.75,0.717096


In [37]:
del ratings_df['rating']

In [38]:
# Guardamos el dataset de ratings con la predicción
training_path = os.path.join(os.getcwd(), "../..", "datasets", "training")
ratings_df.to_csv(training_path + "/predictions_cf_0_100.tsv", sep='\t', header=None, index=False)

---
### Muestra reducida para Elliot

A la hora de ejecutar Elliot con la totalidad de los usuarios (52371), el programa terminaba forzosamente y no se podía completar el experimento para obtener las métricas. Por ello, se ha decidido tomar una muestra reducida de usuarios para computar las distintas métricas del sistema recomendador. El número reducido de usuarios se ha escogido basándonos en los ejemplos disponibles en la documentación de la librería.

In [18]:
import os
import pandas as pd

# Ruta de datasets de entrenamiento
training_path = os.path.join(os.getcwd(), "../..", "datasets", "training")

# Lectura del dataset de predicciones procesado
col_names = ['user_id', 'book_id', 'rating']
col_types = {
    'user_id': 'int32',
    'book_id': 'int32',
    'rating': 'float32',
}
predictions_df = pd.read_csv(training_path + "/predictions/predictions_cb_0_100.tsv", sep='\t', names=col_names, dtype=col_types)
print("# de predicciones:", predictions_df.shape[0])

# de predicciones: 2618550


In [2]:
# IDs de usuarios
user_ids = predictions_df['user_id'].unique()
print("# de usuarios:", len(user_ids))

# de usuarios: 52371


In [3]:
# Selección de num_users usuarios aleatorios
import random

random.seed(42)
num_users = 10000
user_ids_reduced = random.sample(list(user_ids), num_users)
print("# de usuarios:", len(user_ids_reduced))

# de usuarios: 10000


In [19]:
# Obtenemos las predicciones de los usuarios seleccionados
predictions_reduced_df = predictions_df[predictions_df['user_id'].isin(user_ids_reduced)]
print("# de predicciones (reducido):", predictions_reduced_df.shape[0])

# de predicciones (reducido): 500000


In [5]:
# Lectura del dataset de entrenamiento
train_df = pd.read_csv(training_path + "/train.tsv", sep='\t', names=col_names, dtype=col_types)
print("# de ratings en train:", train_df.shape[0])

# de ratings en train: 4412481


In [6]:
# Obtención de los ratings de los usuarios seleccionados
train_reduced_df = train_df[train_df['user_id'].isin(user_ids_reduced)]
print("# de ratings en train (reducido):", train_reduced_df.shape[0])

# de ratings en train (reducido): 842727


In [7]:
# Lectura del dataset de test
test_df = pd.read_csv(training_path + "/test.tsv", sep='\t', names=col_names, dtype=col_types)
print("# de ratings en test:", test_df.shape[0])

# de ratings en test: 1103121


In [8]:
# Obtención de los ratings de los usuarios seleccionados
test_reduced_df = test_df[test_df['user_id'].isin(user_ids_reduced)]
print("# de ratings en test (reducido):", test_reduced_df.shape[0])

# de ratings en test (reducido): 210647


In [ ]:
# Lectura del dataset de train (train \ validation)
train_train_df = pd.read_csv(training_path + "/train_train.tsv", sep='\t', names=col_names, dtype=col_types)
print("# de ratings en train (train + validation):", train_train_df.shape[0])

# de ratings en train (train + validation): 3860920


In [ ]:
# Obtención de los ratings de los usuarios seleccionados
train_train_reduced_df = train_train_df[train_train_df['user_id'].isin(user_ids_reduced)]
print("# de ratings en train (train + validation) (reducido):", train_train_reduced_df.shape[0])

# de ratings en train (train + validation) (reducido): 737597


In [ ]:
# Lectura del dataset de validación
train_valid_df = pd.read_csv(training_path + "/train_valid.tsv", sep='\t', names=col_names, dtype=col_types)
print("# de ratings en validation:", train_valid_df.shape[0])

# de ratings en validation: 551561


In [ ]:
# Obtención de los ratings de los usuarios seleccionados
train_valid_reduced_df = train_valid_df[train_valid_df['user_id'].isin(user_ids_reduced)]
print("# de ratings en validation (reducido):", train_valid_reduced_df.shape[0])

# de ratings en validation (reducido): 105130


In [20]:
# Guardamos el dataset de predicciones así como los de entrenamiento, test y validación
predictions_reduced_df.to_csv(training_path + "/predictions/predictions_reduced_cb_0_100.tsv", sep='\t', header=None, index=False)

In [ ]:
train_reduced_df.to_csv(training_path + "/train_reduced.tsv", sep='\t', header=None, index=False)
test_reduced_df.to_csv(training_path + "/test_reduced.tsv", sep='\t', header=None, index=False)
train_train_df.to_csv(training_path + "/train_train_reduced.tsv", sep='\t', header=None, index=False)
train_valid_df.to_csv(training_path + "/train_valid_reduced.tsv", sep='\t', header=None, index=False)

In [16]:
# Lectura del dataset de predicciones (collaborative filtering)
predictions_cf_df = pd.read_csv(training_path + "/predictions/predictions_cf_0_100.tsv", sep='\t', names=col_names, dtype=col_types)
print("# de predicciones:", predictions_cf_df.shape[0])

# de predicciones: 5515602


In [17]:
# Obtenemos las predicciones de los usuarios seleccionados
predictions_reduced_cf_df = predictions_cf_df[predictions_cf_df['user_id'].isin(user_ids_reduced)]
print("# de predicciones (reducido):", predictions_reduced_cf_df.shape[0])

# de predicciones (reducido): 1053374


In [18]:
# Guardamos el dataset de predicciones (collaborative filtering)
predictions_reduced_cf_df.to_csv(training_path + "/predictions/predictions_reduced_cf_0_100.tsv", sep='\t', header=None, index=False)

-----
### Evaluación manual (sin Elliot)

Vamos a comprobar cuántos de los libros propuestos como más cercanos a los usuarios (es decir, a su perfil) son propuestas que los usuarios escogerían. 

Para ello, consideraremos como un acierto que al menos uno de los libros que el modelo recomienda esté dentro de la lista de libros positivamente valorados por los usuarios en el dataset de test. La primera métrica es el *hit rate*: la proporción de aciertos entre todos los usuarios en el dataset de test.

In [ ]:
test = pd.read_csv("../../datasets/training/test.tsv", sep='\t', header=None)
test.columns = ['user_id', 'book_id', 'rating']

In [ ]:
predictions = pd.read_csv("../../datasets/training/predictions.tsv", sep='\t', header=None)
predictions.columns = ['user_id', 'book_id', 'rating']

In [ ]:
# Calculamos el hit rate
h = 0
user_ids = test['user_id'].unique()
for user_id in user_ids:
    test_user = test.loc[test['user_id'] == user_id]
    predictions_user = predictions.loc[predictions['user_id'] == user_id]
    user_books = test_user.loc[:, 'book_id'].tolist()
    user_likes = test_user.loc[test_user['rating'] >= 0.75, 'book_id'].tolist()
    recommended_books = predictions_user.loc[:, 'book_id'].tolist()
    h += any([b in user_likes for b in recommended_books])
print(h/len(user_ids))

0.07870918464769906


Como vemos, la razón de éxito, para $k = 50$ libros y un peso $w = 0.75$, está en torno al 8%.